# Training Data Processing
This script is used to create a training set, it includes 2 steps:
- Data Cleaning
- Select evidence for NEI case, as the original dataset don't contain evidence for NEI sample

The input for this script is a Retrieval Result that select top 5 sentences in top 5 paragraphs:
- Input file is placed in: ../result/retrieval_result/public_train_retrieval_v1_top5_top_5.json
- The output is placed in: ../result/processed_data/

In [ ]:
import json
import numpy as np
from difflib import SequenceMatcher

In [ ]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [ ]:
INPUT_FILE = ""
OUTPUT_FILE = ""

In [ ]:
data = json.load(open(INPUT_FILE))

In [ ]:
for key in data.keys():
    try:
        if data[key]['verdict'] != 'NEI':
            data[key]['evidence_similarity'] = similar(data[key]['claim'], data[key]['evidence'])
        else:
            data[key]['evidence_similarity'] = 0
        data[key]['retrieval_evidence_similarity'] = list()
        for evidence in data[key]['retrieval_evidence']:
            data[key]['retrieval_evidence_similarity'].append(similar(data[key]['claim'], evidence))
    except:
        print(key)

In [ ]:
new_train_set = dict()
for key in data.keys():
    if ((data[key]['verdict'] == 'SUPPORTED' and data[key]['evidence'] not in data[key]['retrieval_evidence']) or 
       (data[key]['verdict'] == 'REFUTED'and data[key]['evidence'] not in data[key]['retrieval_evidence'])):
            continue
    new_train_set[key] = dict()
    new_train_set[key]['context'] = data[key]['context']
    new_train_set[key]['claim'] = data[key]['claim']
    new_train_set[key]['verdict'] = data[key]['verdict']
    
    if data[key]['verdict'] == "NEI":
        new_train_set[key]['evidence'] = data[key]['retrieval_evidence'][np.argmax(data[key]['retrieval_evidence_similarity'])]
    else:
        new_train_set[key]['evidence'] = data[key]['evidence']

In [ ]:
# Save New Result
with open("public_train_v4.json", "w") as outfile:
    json.dump(new_train_set, outfile)